References:

 - [Fine-tuning BERT for text classification Medium post](https://luv-bansal.medium.com/fine-tuning-bert-for-text-classification-in-pytorch-503d97342db2)
 - [BERT text classification Github jupyter notebook](https://github.com/Nielspace/BERT/blob/master/BERT%20Text%20Classification%20fine-tuning.ipynb)
 - [BERT for sequence classification github](https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/CS546/course-project/SemEval

/content/drive/MyDrive/CS546/course-project/SemEval


In [ ]:
# unzip dataset
def util_unzip_file(path_to_zip_file, directory_to_extract_to):
    import zipfile
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

### Prepare data in csv format

In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report as report

def make_dataframe(input_folder, labels_folder=None):
    #MAKE TXT DATAFRAME
    text = []
    
    for fil in tqdm(filter(lambda x: x.endswith('.txt'), os.listdir(input_folder))):

        iD, txt = fil[7:].split('.')[0], open(input_folder + '/' + fil, 'r', encoding='utf-8').read() 
        text.append((iD, txt))

    df_text = pd.DataFrame(text, columns=['id','text']).set_index('id')

    df = df_text

    #MAKE LABEL DATAFRAME
    if labels_folder:
        labels = pd.read_csv(labels_folder, sep='\t', header=None)
        labels = labels.rename(columns={0:'id',1:'type'})
        labels.id = labels.id.apply(str)
        labels = labels.set_index('id')

        #JOIN
        df = labels.join(df_text)[['text','type']]

    return df

def data_to_csv():
    folder_train = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask1/train-articles-subtask-1"
    folder_dev = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask1/dev-articles-subtask-1"
    labels_train_fn = "/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask1/train-labels-subtask-1.txt"

    #Read Data
    print('Loading training...')
    train = make_dataframe(folder_train, labels_train_fn)
    print('Loading dev...')
    test = make_dataframe(folder_dev)

    # lable encoding: creating a instance of label Encoder.
    label_encoder = LabelEncoder()

    # Using .fit_transform function to fit label
    # encoder and return encoded label
    # opinion: 0, reporting: 1, satire: 2
    label = label_encoder.fit_transform(train['type'])
    train["label"] = label.astype('long')

    train.to_csv("/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask1/train.csv")
    test.to_csv("/content/drive/MyDrive/CS546/course-project/SemEval/data/data_en_subtask1/test.csv")

    return train, test

train, test = data_to_csv()

Loading training...


433it [00:00, 667.15it/s]


Loading dev...


83it [00:00, 1024.59it/s]
